In [1]:
from survival_analysis import *
import pandas as pd

df = prepare_df()

df_analysis = df.copy()

# Add age column
df_analysis['age'] = ((pd.to_datetime("2023-01-01")) - df_analysis['Entry Date']).dt.days / 365.25

# Add dead column
df_analysis['Dead'] = 1 - df_analysis['status']

# Drop ages > 30
df_analysis = df_analysis[df_analysis['age'] <= 30]

df_analysis.head()

,uen,Sector,status,Exit Date,Entry Date,Region,age,Dead
0,53376145W,N,0,2020-04-30 09:36:00,2018-01-13 10:32:56,NR,4.963723,1
1,53290857X,S,1,NaT,2015-02-26 12:02:44,CR,7.843943,0
3,200412424K,G,1,NaT,2004-09-28 17:56:29,WR,18.255989,0
4,53396472E,H,0,2020-10-10 09:36:00,2019-04-11 21:20:43,CR,3.723477,1
5,48376900B,S,0,1999-12-24 04:48:00,1993-04-12 00:00:00,NER,29.722108,1


In [2]:
sector_list = ['G', 'M', 'F', 'J', 'K', 'C', 'H', 'S', 'N', 'I', 'P', 'L', 'Q', 'R']
parameters = [
    [0.13660027, 0.03574423, 12.39113424, 4.14356328],
    [0.10877533, 0.0418096, 12.55269306, 4.41391222],
    [0.079990154, 1.00E-10, 26.18237719, 79.99986416],
    [0.13090805, 0.03791174, 11.9429807, 4.05657508],
    [0.070120134, 0.011071032, 17.60063205, 11.71975389],
    [0.10301031, 0.04128293, 9.26045477, 8.13925264],
    [0.190143914, 0.028016019, 6.93767599, 100],
    [0.14058029, 1.00E-10, 12.9535533, 5.1898739],
    [0.12396223, 1.00E-10, 16.4327672, 3.67640026],
    [0.12568692, 0.03447114, 17.44283135, 5.60609428],
    [0.121213526, 0.068684245, 9.44518567, 100],
    [0.074121126, 1.00E-10, 25.77531849, 79.99860496],
    [0.078301599, 0.047197935, 7.79197632, 100],
    [0.132289514, 0.085485775, 8.85732298, 100]
]
sector_params_MLE = dict(zip(sector_list, parameters))

In [ ]:
import matplotlib.pyplot as plt
from two_tail_dip import *

survival_fractions, ages = obtain_survival_fractions(df_analysis, 'Sector', 'G')
totals, survivors = obtain_total_alive_count(df_analysis, 'Sector', 'G')
# mu_lb, mu_ub, K, m, t_e, delta = fit_hill_with_dip(ages, survivors, totals, [0.18, 0.05, 6, 10, 7, 3]).x

initial_guess = [0.1, 0.05, 10, 5, find_dip(df_analysis, 'G'), 1, 5, 2, 0.1]
# bounds = [
#     (0.01, 0.3),   # mu_ub
#     (0.01, 0.15),   # mu_lb
#     (0.1, 30),     # K
#     (0.5, 100),    # m
#     (3, 10),       # t_e
#     (0.1, 20),    # eps0
#     (0.01, 30)      # tau
# ]
# result = minimize(lsq_hill_with_dip, initial_guess, args=(ages, survival_fractions), bounds=bounds)
result = fit_hill_with_dip(ages, survivors, totals, initial_guess)
mu_ub, mu_lb, K, m, t_e, eps0, tau1, tau2, lam = result.x
print("Fitted parameters (with dip):")
print(f"mu_ub: {mu_ub}, mu_lb: {mu_lb}, K: {K}, m: {m}, t_e: {t_e}, eps0: {eps0}, tau1: {tau1}, tau2: {tau2}, lam: {lam}")

aic_with_dip = 2 * 9 + 2 * neg_ll_hill_with_dip(result.x, ages, survivors, totals)
print(f'AIC = {aic_with_dip}')

plt.plot(ages, survival_fractions, 'o', markersize=3)
plt.plot(ages, model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, eps0, tau1, tau2, lam), label='with dip')
# plt.plot(ages, model_survival_curve_hill(ages, 0.190143914, 0.028016019, 6.93767599, 100), '--', label = 'original MLE fit')
# plt.plot(ages, model_survival_curve_hill(ages, mu_ub, mu_lb, K, m), label='without dip')
plt.title('Sector G fit with dip')
plt.legend()
plt.show()

# AIC for sector G
# with dip: 395498.34404120164, original: 396775.65376669867

NameError: name 'find_dip' is not defined

In [ ]:
import matplotlib.pyplot as plt
from survival_analysis import obtain_survival_fractions, obtain_total_alive_count

survival_fractions, ages = obtain_survival_fractions(df_analysis, 'Sector', 'H')
totals, survivors = obtain_total_alive_count(df_analysis, 'Sector', 'H')
# mu_lb, mu_ub, K, m, t_e, delta = fit_hill_with_dip(ages, survivors, totals, [0.18, 0.05, 6, 10, 7, 3]).x
from scipy.optimize import minimize

initial_guess = [0.1, 0.05, 10, 5, find_dip('H'), 3, 0.5, 1, 0.1]
# bounds = [
#     (0.01, 0.3),   # mu_ub
#     (0.01, 0.15),   # mu_lb
#     (0.1, 30),     # K
#     (0.5, 100),    # m
#     (3, 10),       # t_e
#     (0.1, 20),    # eps0
#     (0.01, 30)      # tau
# ]
# result = minimize(lsq_hill_with_dip, initial_guess, args=(ages, survival_fractions), bounds=bounds)
result = fit_hill_with_dip(ages, survivors, totals, initial_guess)
mu_ub, mu_lb, K, m, t_e, eps0, tau1, tau2, lam = result.x
print("Fitted parameters (with dip):")
print(f"mu_ub: {mu_ub}, mu_lb: {mu_lb}, K: {K}, m: {m}, t_e: {t_e}, eps0: {eps0}, tau1: {tau1}, tau2: {tau2}, lam: {lam}")

aic_with_dip = 2 * 9 + 2 * neg_ll_hill_with_dip(result.x, ages, survivors, totals)
print(f'AIC = {aic_with_dip}')

plt.plot(ages, survival_fractions, 'o', markersize=3)
plt.plot(ages, model_hill_with_dip(ages, mu_ub, mu_lb, K, m, t_e, eps0, tau1, tau2, lam), label='with dip')
# plt.plot(ages, model_survival_curve_hill(ages, 0.190143914, 0.028016019, 6.93767599, 100), '--', label = 'original MLE fit')
# plt.plot(ages, model_survival_curve_hill(ages, mu_ub, mu_lb, K, m), label='without dip')
plt.title('Sector H fit with dip')
plt.legend()
plt.show()